Copyright (C) Microsoft Corporation.

# Load Model and Detect Out of Stock

In this notebook, we adapt the inferancing steps provided [here](https://github.com/tensorflow/models/blob/master/research/object_detection/object_detection_tutorial.ipynb) to detect out-of-stock in the validation set and save the results.

In [ ]:
import numpy as np
import os
import sys
import tensorflow as tf
import glob

from distutils.version import StrictVersion
from collections import defaultdict
from PIL import Image
from utilities import create_dir, load_image_into_numpy_array
from IPython.display import display

from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

if StrictVersion(tf.__version__) < StrictVersion('1.12.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.12.*.')
print(tf.__version__)

# Model preparation 

## Variables

We're going to load the load the frozen graph of the model we trained into memory, and run some images through it.

In [ ]:
test_output = './test_output'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
path_to_frozen_graph = './model/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
path_to_labels = './trainingcode/outofstock_label_map.pbtxt'

data_dir = './dataset/'

In [ ]:
create_dir(test_output)

## Load a (frozen) Tensorflow model into memory.

In [ ]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `1`, we know that this corresponds to `outofstock`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

# Detection

In [ ]:
import random
import glob
random.seed(4)

In [ ]:
in_images = glob.glob(data_dir + '/**/'+ '*.jpg')
print("There are {} images.".format(len(in_images)))

In [ ]:
num_val = int(len(in_images) * 0.05)
val_images = random.sample(in_images, num_val)

In [ ]:
len(val_images)

In [ ]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

In [ ]:
# Size, in inches, of the output images.
IMAGE_SIZE = (24, 18)

In [ ]:
%%time

for image_path in val_images:
    image = Image.open(image_path)
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = load_image_into_numpy_array(image)
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    # Actual detection.
    output_dict = run_inference_for_single_image(image_np, detection_graph)
    print(output_dict['detection_scores'])
    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        min_score_thresh=.02,
        instance_masks=output_dict.get('detection_masks'),
        use_normalized_coordinates=True,
        line_thickness=4)
    image_np = Image.fromarray(image_np)
    save_path = os.path.split(image_path)[-1]
    image_np.save(os.path.join(test_output, save_path))
    print(save_path)

Let's check some of these images.

In [ ]:
val_images_detect = glob.glob(OUT_IMAGES + '/*.jpg')

In [ ]:
for img_path in val_images_detect:
    img = Image.open(img_path)
    display(img)
    print(img.size)